# Librerias

In [1]:
# webdriver ayuda a realizar acciones de navegación en el navegador
from selenium import webdriver
# time ayuda a poder detener el script un tiempo determinado
import time
# Para saber donde estoy :v
import os
#Librerias para trabajar los datos
import pandas as pd
import numpy as np
# Para trabajar con archivos zip
from zipfile import ZipFile

# Funciones

In [2]:
# Función para descargar los archivos de los municipios
def descargar_archivos(lista):
    
    """
    Configuración opciones del webdriver
    """
    
    # Ayuda a establecer las preferencias del navegador chrome
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': os.getcwd() + '\\data_source'}
    #Permite añadir nuestras preferencias al driver
    options.add_experimental_option("prefs",prefs)
    
    """
    Configuración webdriver
    """
    # Ruta del driver
    PATH = os.getcwd() + '\\chromedriver\\chromedriver.exe'
    # Creamos el driver
    driver = webdriver.Chrome(executable_path=PATH, options = options)
    
    """
    Ahora si la descarga
    """
    
    for muni in lista:        
        file_name = f'./data_source/TerriData{muni}f.xlsx.zip'
        # Para cada municipio, si este zip no esta en "./data_source/", se descarga
        if not os.path.isfile(file_name):
            print('descargando municipio: ' + str(muni))
            try:
                # Buscamos el objeto
                driver.get(f'https://terridata.dnp.gov.co/index-app.html#/perfiles/{muni}')
                time.sleep(2)
                downloadxlsx = driver.find_element_by_id('btnExcel')
                downloadxlsx.click()
            except Exception as e:
                print(str(e))
            # Esperamos 30 segundos porque no quiero que la página banee mi ip
            time.sleep(8)
        print(f'Municipio {muni} descargado.')
        
    #Cerramos el webdriver
    driver.quit()
        

In [3]:
# Arregla los tipos del dataframe
def arreglar_tipos(df:pd.DataFrame, types:dict) -> None:
    for value in types:
        try:
            df[value] = df[value].astype(types[value])
        except Exception as e:
            print(f'error in value {value}',str(e))

# Trabajo con el Dataframe
## Paso 1
Traigo el dataframe y lo limpio

In [4]:
# Dataframe central
df_main = pd.read_csv('data_source/data_2018_2020.csv',sep=',',index_col=False,low_memory=False)

# Limpieza
df_main = df_main.drop(df_main[df_main['codigo_depto']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['nombre_depto']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['nombre_muni']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['codigo_muni']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['codigo_evento']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['evento']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['edad']=='TOTAL'].index,errors='ignore')
df_main = df_main.drop(df_main[df_main['sexo']=='TOTAL'].index,errors='ignore')

# Lo de los extranjeros
df_main['codigo_depto'] = df_main['codigo_depto'].apply(lambda x : 100 if x == 'Extranjeros' else x)

# Quitamos muertes en el extranjero (columna codigo_depto con el valor 100)
df_main = df_main.drop(df_main[df_main['codigo_depto']==100].index)
# Quitamos sexo indeterminado (columna sexo con el valor "Indeterminado")
df_main = df_main.drop(df_main[df_main['sexo']=='Indeterminado'].index)
# Quitamos muertes de edad desconocida (columna edad con valor "Edad Desconocida")
df_main = df_main.drop(df_main[df_main['edad']=='Edad desconocida'].index)

"""
En la columna edad quitaremos los valores correspondientes a "De 65 y más" dado que en esa columna a 
la vez hay rangos como "De 65-84 años", "De 85-99 años", "De 100 y más" y al ser dos versiones de la 
misma información (aparentemente), decidimos quedarnos con los otros rangos en espera de que nos den 
más info.
"""
df_main = df_main.drop(df_main[df_main['edad']=='De 65 y más'].index)

# Arreglamos los tipos
value_types = {'anio':'int64','codigo_depto':'string','nombre_depto':'string',
               'nombre_muni':'string','codigo_muni':'string','codigo_evento':'int64',
               'evento':'string','sexo':'string','edad':'string','num_casos':'int64'
              }
arreglar_tipos(df_main,value_types)

# Hay que arreglar los rangos de las edades
rangos = {'Menor 1 año':'0-4 años','De 1-4 años':'0-4 años','De 5-14 años':'5-14 años',
          'De 15-44 años':'15-44 años','De 45-64 años':'45-64 años','De 65-84 años':'65+ años',
          'De 85-99 años':'65+ años','De 100 y más':'65+ años'
         }

df_main['edad'] = df_main['edad'].apply(lambda x : rangos[x])

#Los valores que nos hayan quedado iguales deben ser sumados
group_by_parameters = ['anio','codigo_depto','nombre_depto','nombre_muni','codigo_muni',
                       'codigo_evento','evento','edad','sexo']
df_main = df_main.groupby(by=group_by_parameters).sum()
#Para volverlo dataframe denuevo
df_main = df_main.reset_index()

# Imprimo
df_main.head()

,anio,codigo_depto,nombre_depto,nombre_muni,codigo_muni,codigo_evento,evento,edad,sexo,num_casos
0,2018,05,Antioquia,Abejorral,05002,102,Tuberculosis,65+ años,Hombres,1
1,2018,05,Antioquia,Abejorral,05002,106,"Septicemia, excepto neonatal",65+ años,Mujeres,2
2,2018,05,Antioquia,Abejorral,05002,108,Infecciones respiratorias agudas,0-4 años,Mujeres,1
3,2018,05,Antioquia,Abejorral,05002,108,Infecciones respiratorias agudas,65+ años,Hombres,4
4,2018,05,Antioquia,Abejorral,05002,108,Infecciones respiratorias agudas,65+ años,Mujeres,3


## Paso 2
Saco todos los códigos de municipios

In [6]:
# Aqui saco solo el Caquetá por el test
df_main = df_main[(df_main["nombre_depto"] == "Caquetá")
                 | (df_main["nombre_depto"] =="Antioquia")
                 | (df_main["nombre_depto"] =="Boyacá")
                 | (df_main["nombre_depto"] =="Chocó")
                 | (df_main["nombre_depto"] =="Huila")
                 ]

In [7]:
# Me quedo con los códigos
muni_list = df_main["codigo_muni"].unique()
muni_list

array(['05002', '05004', '05021', '05030', '05031', '05034', '05038',
       '05040', '05044', '05045', '05051', '05055', '05059', '05079',
       '05088', '05086', '05091', '05093', '05107', '05113', '05125',
       '05129', '05134', '05142', '05145', '05147', '05150', '05154',
       '05138', '05172', '05190', '05101', '05197', '05206', '05209',
       '05212', '05120', '05234', '05237', '05240', '05250', '05148',
       '05697', '05264', '05266', '05282', '05284', '05306', '05308',
       '05313', '05315', '05318', '05321', '05310', '05347', '05353',
       '05360', '05361', '05364', '05368', '05376', '05380', '05390',
       '05400', '05411', '05425', '05440', '05001', '05467', '05475',
       '05480', '05483', '05495', '05490', '05501', '05543', '05541',
       '05576', '05579', '05585', '05591', '05604', '05607', '05615',
       '05628', '05631', '05642', '05647', '05649', '05652', '05656',
       '05658', '05659', '05660', '05664', '05665', '05667', '05670',
       '05674', '056

## Paso 3
Aqui realizo la descarga del zip, si no tengo el archivo descargado, lo descargo

In [8]:
descargar_archivos(muni_list)

Municipio 05002 descargado.
Municipio 05004 descargado.
Municipio 05021 descargado.
Municipio 05030 descargado.
Municipio 05031 descargado.
Municipio 05034 descargado.
Municipio 05038 descargado.
Municipio 05040 descargado.
Municipio 05044 descargado.
Municipio 05045 descargado.
Municipio 05051 descargado.
Municipio 05055 descargado.
Municipio 05059 descargado.
Municipio 05079 descargado.
Municipio 05088 descargado.
Municipio 05086 descargado.
Municipio 05091 descargado.
Municipio 05093 descargado.
Municipio 05107 descargado.
Municipio 05113 descargado.
Municipio 05125 descargado.
Municipio 05129 descargado.
Municipio 05134 descargado.
Municipio 05142 descargado.
Municipio 05145 descargado.
Municipio 05147 descargado.
Municipio 05150 descargado.
Municipio 05154 descargado.
Municipio 05138 descargado.
Municipio 05172 descargado.
Municipio 05190 descargado.
Municipio 05101 descargado.
Municipio 05197 descargado.
Municipio 05206 descargado.
Municipio 05209 descargado.
Municipio 05212 desc

## Paso 4
Creo una variable nueva df_ready, que ca a ir concatenando los resultados de cada iteración, ahora, para cada archivo:
+ Extraigo el zip
+ Traigo su dataframe y lo guardo en una variable "temp"
+ Saco lo que me interesa y realizo limpieza
+ Junto con df_main con un join para mezclar la información
+ Concateno con los datos ya juntados anteriormente en df_ready
+ Borro el archivo que salga del zip

In [9]:
df_ready = None
first_time = False

for muni in muni_list:
    filename = f'./data_source/TerriData{muni}f.xlsx'
    with ZipFile((filename+'.zip'), 'r') as zip:
        #zip.printdir()
        zip.extractall('./data_source/')
        
    # Cargo el archivo
    value_types = {'Código Departamento':'string','Código Entidad':'string'}
    temp = pd.read_excel(filename,dtype=value_types)

    # Me quedo solo con lo que me interesa
    temp = temp[(temp["Dimensión"] == "Demografía y población") 
                & 
                (
                    (temp["Subcategoría"] == "Población de hombres") 
                    |
                    (temp["Subcategoría"] == "Población de mujeres")
                )
               ]

    # Solo conservo las columnas que quiero
    temp = temp.drop(columns=['Dimensión','Subcategoría','Dato Cualitativo','Mes','Fuente'],errors='ignore')

    # Arreglo los nombres de las columnas
    temp.columns = ['codigo_depto','nombre_depto','codigo_muni','nombre_muni','edad','población','anio','sexo']

    # Toca cambiar los '.' por ',' en la columna población  y visceversa 
    temp['población'] = temp['población'].apply(lambda x : x.replace('.',''))
    temp['población'] = temp['población'].apply(lambda x : x.replace(',','.'))
    temp['población'] = temp['población'].astype('float64')

    # Arreglamos los tipos
    value_types = {'codigo_depto':'string','nombre_depto':'string','codigo_muni':'string',
                   'nombre_muni':'string','edad':'string','población':'int64',
                   'anio':'int64','sexo':'string'
                  }

    arreglar_tipos(temp,value_types)

    # Hay que arreglar los rangos de las edades
    rangos_temp = {'Población de hombres de 00-04':'0-4 años','Población de hombres de  05-09':'5-14 años',
                 'Población de hombres de  10-14':'5-14 años','Población de hombres de  15-19':'15-44 años',
                 'Población de hombres de  20-24':'15-44 años','Población de hombres de  25-29':'15-44 años',
                 'Población de hombres de  30-34':'15-44 años','Población de hombres de  35-39':'15-44 años',
                 'Población de hombres de  40-44':'15-44 años','Población de hombres de  45-49':'45-64 años',
                 'Población de hombres de  50-54':'45-64 años','Población de hombres de  55-59':'45-64 años',
                 'Población de hombres de  60-64':'45-64 años','Población de hombres de  65-69':'65+ años',
                 'Población de hombres de  70-74':'65+ años','Población de hombres de  75-79':'65+ años',
                 'Población de hombres de 80 o más':'65+ años','Población de mujeres de 00-04':'0-4 años',
                 'Población de mujeres de  05-09':'5-14 años','Población de mujeres de  10-14':'5-14 años',
                 'Población de mujeres de  15-19':'15-44 años','Población de mujeres de  20-24':'15-44 años',
                 'Población de mujeres de  25-29':'15-44 años','Población de mujeres de  30-34':'15-44 años',
                 'Población de mujeres de  35-39':'15-44 años','Población de mujeres de  40-44':'15-44 años',
                 'Población de mujeres de  45-49':'45-64 años','Población de mujeres de  50-54':'45-64 años',
                 'Población de mujeres de  55-59':'45-64 años','Población de mujeres de  60-64':'45-64 años',
                 'Población de mujeres de  65-69':'65+ años','Población de mujeres de  70-74':'65+ años',
                 'Población de mujeres de  75-79':'65+ años','Población de mujeres de 80 o más':'65+ años'
                }

    temp['edad'] = temp['edad'].apply(lambda x : rangos_temp[x])

    #Los valores que nos hayan quedado iguales deben ser sumados
    temp_group_by_parameters_1 = ['codigo_depto','nombre_depto','codigo_muni',
                                  'nombre_muni','edad','anio','sexo'
                                 ]
    temp = temp.groupby(by=temp_group_by_parameters_1).sum()
    #Para volverlo dataframe denuevo
    temp = temp.reset_index()
    
    # Juntamos con el dataframe original
    temp_group_by_parameters_2 = ['codigo_depto','nombre_depto','codigo_muni',
                                  'nombre_muni','edad','anio','sexo'
                                 ]
    
    temp_merged = pd.merge(df_main, temp,  on=temp_group_by_parameters_2)
    if first_time:
        df_ready = temp_merged
        first_time = False
    else:
        df_ready = pd.concat([df_ready, temp_merged])
    
    os.remove(filename)

# Visualizamos algún dataframe temporal
temp.head()

,codigo_depto,nombre_depto,codigo_muni,nombre_muni,edad,anio,sexo,población
0,41,Huila,41885,Yaguará,0-4 años,2020,Hombres,310
1,41,Huila,41885,Yaguará,0-4 años,2020,Mujeres,323
2,41,Huila,41885,Yaguará,15-44 años,2020,Hombres,1744
3,41,Huila,41885,Yaguará,15-44 años,2020,Mujeres,1617
4,41,Huila,41885,Yaguará,45-64 años,2020,Hombres,902


In [10]:
# Visualizamos el dataframe actualmente
df_ready.head()

,anio,codigo_depto,nombre_depto,nombre_muni,codigo_muni,codigo_evento,evento,edad,sexo,num_casos,población
0,2020,05,Antioquia,Abejorral,05002,102,Tuberculosis,15-44 años,Hombres,1,4393
1,2020,05,Antioquia,Abejorral,05002,501,Accidentes de transporte terrestre,15-44 años,Hombres,1,4393
2,2020,05,Antioquia,Abejorral,05002,511,Lesiones autoinfligidas intencionalmente (suic...,15-44 años,Hombres,1,4393
3,2020,05,Antioquia,Abejorral,05002,512,Agresiones (homicidios),15-44 años,Hombres,1,4393
4,2020,05,Antioquia,Abejorral,05002,108,Infecciones respiratorias agudas,65+ años,Hombres,1,1358


In [11]:
df_ready.to_csv('data_generated/data_ready_test.csv', index = False)